In [1]:
#### THIS MODEL USES CS109's titanic dataset to learn and then predict whether you'd survive if you were on board the titanic in 1912.

In [2]:
%matplotlib inline

#importing classes
import copy
import numpy as np 
import pandas as pd 
import sklearn.model_selection as skm 
import matplotlib.pyplot as plt 
import json

In [3]:
data=pd.read_csv("DataSets/titanic.csv")
y=data.Survived
x=data.drop(['Name','Survived'],axis=1)
x.loc[x.Sex=="male","Sex"]=1
x.loc[x.Sex=="female","Sex"]=0
x_train,x_test,y_train,y_test=[x.to_numpy() for x in skm.train_test_split(x,y,test_size=0.2)]
print(x_train.shape)
#offloading data from memory
del x
del y
del data

#using sklearn.model_selection we split the databases in test and train in 20%-80% ratio.

(709, 6)


In [4]:
#Reshaping the data to fit the proper value definitions:
x_train=x_train.reshape(x_train.shape[0],-1).T
x_test=x_test.reshape(x_test.shape[0],-1).T
y_train=y_train.reshape(y_train.shape[0],-1).T
y_test=y_test.reshape(y_test.shape[0],-1).T

x_train=x_train/x_train.std()
x_test=x_test/x_test.std()
y_test=y_test/y_test.std()
y_train=y_train/y_train.std()

In [5]:
#Actual development starts here
#normal base functions 

In [32]:
#sigmoid function
def sigmoid(z):
    z=z.astype(float)
    s=1/(1+np.exp(-z))
    return s

In [33]:
#initializer
def init(dim):
    w=np.zeros((dim,1))
    b=0.0
    return w,b

In [37]:
#propagation function used for forward and backward propagation
def propagate(w,b,X,Y):
    #forward pass 

    #creation of activation function
    m=X.shape[1]
    A=sigmoid(np.dot(w.T,X)+b) 

    #calculation of cost function

    cost=-(1/m)*np.sum(((Y*np.log(A))+((1-Y)*np.log(1-A))))


    #Backward Propagation: to find grad
    dw=(np.dot(X,(A-Y).T))/m
    db=(np.sum(A-Y))/m

    cost=np.squeeze(np.array(cost))

    grads={
        "dw":dw,
        "db":db
    }

    return grads,cost

In [41]:
#optimization function to run epochs and use transfer learning to update variables

def optimize(w,b,X,Y,epochs=1000,learning_rate=0.5,view_cost=True):
    costs=np.asarray([])    #np array to save costs
    
    for x in range(epochs):
        grads,cost=propagate(w,b,X,Y)  #calculates 1 epoch and copies grads and cost here

        #updating variables
        
        dw=grads["dw"]
        db=grads["db"]

        w=w-(learning_rate * dw) #the gradient is now used to updated
        b=b-(learning_rate * db)

        #printing out the cost function if view_cost=True:
        if view_cost:
            if x%200==0:
                print("The cost after ",x,"iterations is : ",cost)
        
    #storing the values

    params={
        "w":w,
        "b":b
    }

    grads={
        "dw":dw,
        "db":db
    }
#returning the values after n epochs
    return params,grads,costs


In [49]:
#this is the part where we try to predict the value

def predict(w,b,X):
    print(X.shape)
    m=X.shape[1]
    Y_predict=np.zeros((1,m))
    #w=w.reshape(1,X.shape[1])

    #reshaping and initializing the values
    A=sigmoid(np.dot(w.T,X)+b)

    #vectorized implementation : 

    Y_predict=(A>=0.5)*1.0

    return Y_predict

In [28]:
#final Function used to call all the values

#The training is by default true , if its not training then set trainning = false during function call

def model(X_train,Y_train,X_test,Y_test,epochs=1000,learning_rate=0.5,print_cost=True):
    w,b=init(X_train.shape[0])

    params,grads,costs=optimize(w,b,X_train,Y_train,epochs=epochs,learning_rate=learning_rate,view_cost=print_cost)

    w=params["w"]
    b=params["b"]

    Y_predict_test=predict(w,b,Y_test)
    Y_predict_train=predict(w,b,Y_train)

    if(print_cost):
        print("The Accuracy on test set: ",(100-np.mean(np.abs(Y_predict_test-Y_test))*100),"%")
        print("The Accuracy on train set: ",(100-np.mean(np.abs(Y_predict_train-Y_train))*100),"%")

    d={
        "w":w,
        "b":b
        }
    with open("learning_parameter.json","w") as write:
        json.dumps(d,write)
    return d

In [50]:
regression=model(x_train,y_train,x_test,y_test,epochs=2000,learning_rate=0.005,print_cost=True)

The cost after  0 iterations is :  0.6931471805599453
The cost after  200 iterations is :  0.341256317384601
The cost after  400 iterations is :  0.1973188565550425
The cost after  600 iterations is :  0.0834152396287583
The cost after  800 iterations is :  -0.01988867166529872
The cost after  1000 iterations is :  -inf
<ipython-input-37-8f386d4603cd>:11: RuntimeWarning: divide by zero encountered in log
  cost=-(1/m)*np.sum(((Y*np.log(A))+((1-Y)*np.log(1-A))))
The cost after  1200 iterations is :  -inf
The cost after  1400 iterations is :  -inf
The cost after  1600 iterations is :  -inf
The cost after  1800 iterations is :  -inf
(1, 178)


ValueError: shapes (1,6) and (1,178) not aligned: 6 (dim 1) != 1 (dim 0)